In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:

import tensorflow.keras.losses as losses
import tensorflow.keras.optimizers as optimizers
from tensorflow.keras.models import load_model
import numpy as np
from PIL import Image

# Example weights for each model
weights = {
    "DayNightSense": 1/8,
    "FogVision": 1/8,
    "MultiLabelModel": 2/8,  # Adjusted weight for the multi-label model
    "Skyfall": 2/8  # Adjusted weight for the new multi-label model
}

# Placeholder for combined result categories
categories = ["day", "night", "fog", "clear", "Sandstrom", "Sunny", "Thunderstrom", "Tornado", "Rain", "Snow", "Sunny"]

# Load and compile each model
day_night_model = load_model(
    '/content/drive/MyDrive/Final_Year_Weather_Classification/CNN_Resuidal_Code/Entire_CNN_DayNight_BinaryModel.h5',
    compile=True
)
day_night_model.compile(
    optimizer=optimizers.Adam(),
    loss=losses.BinaryCrossentropy(),
    metrics=['accuracy']
)

fog_model = load_model(
    '/content/drive/MyDrive/Final_Year_Weather_Classification/CNN_Resuidal_Code/Entire_CNN_FogNet_BinaryModel.h5',
    compile=True
)
fog_model.compile(
    optimizer=optimizers.Adam(),
    loss=losses.BinaryCrossentropy(),
    metrics=['accuracy']
)

multi_label_model = load_model(
    '/content/drive/MyDrive/Final_Year_Weather_Classification/CNN_Resuidal_Code/My_Strom_Vision.keras',
    compile=True
)
multi_label_model.compile(
    optimizer=optimizers.Adam(),
    loss=losses.BinaryCrossentropy(),
    metrics=['accuracy']
)

skyfall_model = load_model(
    '/content/drive/MyDrive/Final_Year_Weather_Classification/CNN_Resuidal_Code/My_SkyFall.keras',
    compile=True
)
skyfall_model.compile(
    optimizer=optimizers.Adam(),
    loss=losses.BinaryCrossentropy(),
    metrics=['accuracy']
)

# Function to normalize model outputs to sum to 1
def normalize(probs):
    total = sum(probs.values())
    return {k: v / total for k, v in probs.items()}

# Function to get predictions from each model
def get_predictions(image, threshold=0.5):
    # Get day/night prediction
    day_night_probs = day_night_model.predict(image)[0]
    print(f"Day/Night model output: {day_night_probs}")
    day_night_result = {"day": day_night_probs[0] * 100, "night": (1 - day_night_probs[0]) * 100}

    # Get fog prediction
    fog_probs = fog_model.predict(image)[0]
    print(f"Fog model output: {fog_probs}")
    fog_result = {"fog": fog_probs[0] * 100, "clear": (1 - fog_probs[0]) * 100}

    # Get multi-label predictions
    multi_label_probs = multi_label_model.predict(image)[0]
    print(f"Multi-label model output: {multi_label_probs}")
    multi_label_result = {
        "Sandstrom": multi_label_probs[0] * 100,
        "Sunny": multi_label_probs[1] * 100,
        "Thunderstrom": multi_label_probs[2] * 100,
        "Tornado": multi_label_probs[3] * 100
    }

    # Get Skyfall predictions
    skyfall_probs = skyfall_model.predict(image)[0]
    print(f"Skyfall model output: {skyfall_probs}")
    skyfall_result = {
        "Rain": skyfall_probs[0] * 100,
        "Snow": skyfall_probs[1] * 100,
        "Sunny": skyfall_probs[2] * 100
    }

    model_outputs = {
        "DayNightSense": day_night_result,
        "FogVision": fog_result,
        "MultiLabelModel": multi_label_result,
        "Skyfall": skyfall_result
    }

    return model_outputs

# Function to combine model outputs
def combine_model_outputs(model_outputs, weights):
    combined_result = {category: 0 for category in categories}

    for model, output in model_outputs.items():
        normalized_output = normalize(output)
        for condition, score in normalized_output.items():
            combined_result[condition] += weights[model] * score

    # Normalize the combined result
    total_weight = sum(weights.values())
    combined_result = {condition: score / total_weight for condition, score in combined_result.items()}

    # Convert combined result to percentages
    combined_result = {condition: score * 100 for condition, score in combined_result.items()}

    # Determine final prediction
    final_prediction = max(combined_result, key=combined_result.get)

    return combined_result, final_prediction

# Function to load and preprocess the image
def load_and_preprocess_image(image_path):
    image = Image.open(image_path)
    # Convert to RGB if image has an alpha channel
    if image.mode == 'RGBA':
        image = image.convert('RGB')
    image = image.resize((224, 224))  # Resize to match the input size expected by your models
    image = np.array(image) / 255.0   # Normalize the image
    image = np.expand_dims(image, axis=0)  # Add batch dimension
    return image

# Example usage
image_path = '/content/drive/MyDrive/Final_Year_Weather_Classification/Dataset/SkyFall/Snow/Screenshot (246).png'
image = load_and_preprocess_image(image_path)

model_outputs = get_predictions(image)
combined_result, final_prediction = combine_model_outputs(model_outputs, weights)

print("Combined Result:", {k: f"{v:.2f}%" for k, v in combined_result.items()})
print("Final Prediction:", final_prediction)

/usr/local/lib/python3.10/dist-packages/keras/src/saving/saving_lib.py:576: UserWarning: Skipping variable loading for optimizer 'adam', because it has 10 variables whereas the saved optimizer has 14 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
Day/Night model output: [1.7539445e-14]
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
Fog model output: [0.00080445]
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
Multi-label model output: [0.04477 0.6353  0.249   0.05624]
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
Skyfall model output: [0.3406 0.4602 0.1763]
Combined Result: {'day': '0.00%', 'night': '16.67%', 'fog': '0.01%', 'clear': '16.65%', 'Sandstrom': '1.51%', 'Sunny': '27.50%', 'Thunderstrom': '8.42%', 'Tornado': '1.90%', 'Rain': '11.62%', 'Snow': '15.70%'}
Final Prediction: Sunny


In [2]:
!pip install --upgrade tensorflow

import tensorflow.keras.losses as losses
import tensorflow.keras.optimizers as optimizers
from tensorflow.keras.models import load_model
import numpy as np
from PIL import Image

# ... rest of your code ...

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 589.8/589.8 MB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 64.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 65.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 66.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 57.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 29.2 MB/s eta 0:00:00
  Attempting uninstall: ml-dtypes
    Found existing installation: ml-dtypes 0.2.0
    Uninstalling ml-dtypes-0.2.0:
      Successfully uninstalled ml-dtypes-0.2.0
  Attempting uninstall: h5py
    Found existing installation: h5py 3.9.0
    Uninstalling h5py-3.9.0:
      Successfully uninstalled h5py-3.9.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.15.2
    Uninstalling tensorboard-2.15.2:
      Successfully uninstalled tensorboard-2.15.2
  Attempting uninstall: keras
    Fo

In [3]:
# Function to load and preprocess the image
def load_and_preprocess_image(image_path):
    image = Image.open(image_path).convert('RGB') # Convert image to RGB format
    image = image.resize((224, 224))  # Resize to match the input size expected by your models
    image = np.array(image) / 255.0   # Normalize the image
    image = np.expand_dims(image, axis=0)  # Add batch dimension
    return image

In [4]:
!pip install h5py
import h5py

# Open the HDF5 file
with h5py.File('/content/drive/MyDrive/Final_Year_Weather_Classification/CNN_Resuidal_Code/CNN_DayNight_BinaryModel150ep.h5', 'r') as f:
    # Print the keys (contents) of the file
    print(f.keys())

<KeysViewHDF5 ['conv1_bn', 'conv1_conv', 'conv1_pad', 'conv1_relu', 'conv2_block1_0_bn', 'conv2_block1_0_conv', 'conv2_block1_1_bn', 'conv2_block1_1_conv', 'conv2_block1_1_relu', 'conv2_block1_2_bn', 'conv2_block1_2_conv', 'conv2_block1_2_relu', 'conv2_block1_3_bn', 'conv2_block1_3_conv', 'conv2_block1_add', 'conv2_block1_out', 'conv2_block2_1_bn', 'conv2_block2_1_conv', 'conv2_block2_1_relu', 'conv2_block2_2_bn', 'conv2_block2_2_conv', 'conv2_block2_2_relu', 'conv2_block2_3_bn', 'conv2_block2_3_conv', 'conv2_block2_add', 'conv2_block2_out', 'conv2_block3_1_bn', 'conv2_block3_1_conv', 'conv2_block3_1_relu', 'conv2_block3_2_bn', 'conv2_block3_2_conv', 'conv2_block3_2_relu', 'conv2_block3_3_bn', 'conv2_block3_3_conv', 'conv2_block3_add', 'conv2_block3_out', 'conv3_block1_0_bn', 'conv3_block1_0_conv', 'conv3_block1_1_bn', 'conv3_block1_1_conv', 'conv3_block1_1_relu', 'conv3_block1_2_bn', 'conv3_block1_2_conv', 'conv3_block1_2_relu', 'conv3_block1_3_bn', 'conv3_block1_3_conv', 'conv3_block